In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e12/sample_submission.csv
/kaggle/input/playground-series-s3e12/train.csv
/kaggle/input/playground-series-s3e12/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s3e12/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e12/test.csv')
submission = pd.read_csv('/kaggle/input/playground-series-s3e12/sample_submission.csv')

In [3]:
submission.shape

(276, 2)

In [4]:
train.shape

(414, 8)

In [5]:
test.shape

(276, 7)

In [6]:
train.head()

,id,gravity,ph,osmo,cond,urea,calc,target
0,0,1.013,6.19,443,14.8,124,1.45,0
1,1,1.025,5.40,703,23.6,394,4.18,0
2,2,1.009,6.13,371,24.5,159,9.04,0
3,3,1.021,4.91,442,20.8,398,6.63,1
4,4,1.021,5.53,874,17.8,385,2.21,1


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       414 non-null    int64  
 1   gravity  414 non-null    float64
 2   ph       414 non-null    float64
 3   osmo     414 non-null    int64  
 4   cond     414 non-null    float64
 5   urea     414 non-null    int64  
 6   calc     414 non-null    float64
 7   target   414 non-null    int64  
dtypes: float64(4), int64(4)
memory usage: 26.0 KB


Our life is made easier by all the features having the same amount of entries, however we can see they are all at a very different scale

In [8]:
X = train.drop('target', axis=1)
y = train['target'].copy()

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2)

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
scaler = StandardScaler()
X_scaled =  scaler.fit_transform(X)
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
test_scaled = scaler.transform(test)

In [12]:
X_train_scaled.shape

(331, 7)

In [13]:
from sklearn.svm import LinearSVC

In [14]:
lin_svc = LinearSVC(loss='hinge', C=1)

In [15]:
lin_svc.fit(X_train_scaled, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=1, loss='hinge')

In [16]:
from sklearn.metrics import accuracy_score

y_pred = lin_svc.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.7401812688821753

In [17]:
from sklearn.svm import SVC

In [18]:
svc = SVC(kernel='rbf', gamma=5, C=0.001)

In [19]:
svc.fit(X_train_scaled, y_train)

SVC(C=0.001, gamma=5)

In [20]:
y_pred = svc.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.5589123867069486

In [21]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svc, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END ....C=8.886683066326128, gamma=0.016469970172154763; total time=   0.0s
[CV] END ....C=8.886683066326128, gamma=0.016469970172154763; total time=   0.0s
[CV] END ....C=8.886683066326128, gamma=0.016469970172154763; total time=   0.0s
[CV] END ....C=3.347574506096919, gamma=0.012570890747323484; total time=   0.0s
[CV] END ....C=3.347574506096919, gamma=0.012570890747323484; total time=   0.0s
[CV] END ....C=3.347574506096919, gamma=0.012570890747323484; total time=   0.0s
[CV] END ...C=3.378805283826675, gamma=0.0035162285675212775; total time=   0.0s
[CV] END ...C=3.378805283826675, gamma=0.0035162285675212775; total time=   0.0s
[CV] END ...C=3.378805283826675, gamma=0.0035162285675212775; total time=   0.0s
[CV] END ...C=4.070332350364677, gamma=0.0018017084929360166; total time=   0.0s
[CV] END ...C=4.070332350364677, gamma=0.0018017084929360166; total time=   0.0s
[CV] END ...C=4.070332350364677, gamma=0.0018017

RandomizedSearchCV(cv=3, estimator=SVC(C=0.001, gamma=5),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7e87427a11e0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7e87427a0d30>},
                   verbose=2)

In [22]:
rnd_search_cv.best_estimator_

SVC(C=3.347574506096919, gamma=0.012570890747323484)

In [23]:
rnd_search_cv.best_score_

0.7463008463008464

In [24]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

SVC(C=3.347574506096919, gamma=0.012570890747323484)

In [25]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.7492447129909365

In [26]:
y_pred = rnd_search_cv.best_estimator_.predict(X_val_scaled)
accuracy_score(y_val, y_pred)

0.7831325301204819

In [27]:
y_pred_ = rnd_search_cv.best_estimator_.predict(X_scaled)
accuracy_score(y, y_pred_)

0.7584541062801933

In [28]:
y_pred__ = rnd_search_cv.best_estimator_.predict(test_scaled)

In [29]:
submission_df = pd.DataFrame({'id': test['id'], 'target': y_pred__})
submission_df.to_csv('submission.csv', index=False)